In [ ]:
import os

import numpy as np
import tensorflow as tf
import pandas as pd
from sklearn.preprocessing import LabelEncoder


Columns = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Embarked', 'Fare']

def readData(FileName, is_test=False):
    titanic_df = pd.read_csv(FileName)
    if is_test:
        titanic_df = titanic_df.replace([None], [0])
    else:
        titanic_df = titanic_df[pd.notnull(titanic_df['Age'])]
        titanic_df = titanic_df[pd.notnull(titanic_df['Embarked'])]
    return titanic_df



def processData(data):
    gender = LabelEncoder()
    gender.fit(['male', 'female'])
    data["Sex"]= gender.transform(data["Sex"])

    Embarked = LabelEncoder()
    Embarked.fit(['S', 'C','Q'])
    data["Embarked"] = Embarked.transform(data["Embarked"])  
    return data

def input_fn(data): 
    Continuous_cols = {k: tf.constant(data[k].values) for k in Columns}
    label = tf.constant(data['Survived'].values)
    return dict(Continuous_cols.items()),label



def test_fn(data):
    Continuous_cols = {k: tf.constant(data[k].values) for k in Columns}
    return Continuous_cols
    
def DNN(train_data, test_data):
    features = []
    for c in Columns:
        features.append(tf.contrib.layers.real_valued_column(str(c)))
   
    
    classifier = tf.contrib.learn.DNNClassifier(feature_columns=features,
                                                hidden_units =[10,20,20,10],
                                                n_classes=2)
    
    classifier.fit(input_fn=lambda: input_fn(train_data), steps=10000)
    accuracy = classifier.evaluate(input_fn=lambda: input_fn(train_data), steps=1000)
    
    
    return accuracy,list(classifier.predict(input_fn=lambda:test_fn(test_data)))


   
def main():
    tf.set_random_seed(10)
    test_data = readData("../data/test.csv", True)
    # print(test_data)
    train_data = readData("../data/train.csv", False)

    test_data = processData(test_data)
    train_data = processData(train_data)
    
    # print(test_data)
    accuracy, survived = DNN(train_data, test_data)
    print("\n\n\n")
    for key in sorted(accuracy):
       print("%s: %s" % (key, accuracy[key]*100))
    
    print("\n\n\n")
    passengers = list(test_data['PassengerId'])
    index = 0
    
#     for survivor in survived:
#         print(str(survivor) +"\t:" +str(passengers[index]))
#         index = index +1
    indexes = test_data.index.values
    with open("kaggle.csv", "w") as f:
        f.write("PassengerId,Survived\n")
        for index, prediction in zip(indexes, survived):
            f.write("{0},{1}\n".format(index, prediction))

if __name__ == "__main__":
    main()
